In [2]:
import ase
from ase.io import read, write
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'eval'

In [59]:
repeat = 20
cell_ratio = 1/2

In [60]:
from kmcos.run import KMC_Model
from kmcos.view import main


with KMC_Model(print_rates=False, banner=False) as model:
    model.settings.simulation_size = repeat
    atoms = model.get_atoms()
    converter = {"fe": model.proclist.fe, "empty": model.proclist.empty, "pt": model.proclist.pt}
    for coord, type in zip(merged[["x_cell", "y_cell", "z_cell", "inner_coord_number"]].values, merged["types_y"]):
        model._put(coord.tolist(), converter[type])
    model._adjust_database()
    #model.do_steps(0)
    aft_atoms = model.get_atoms()
  #print(model.proclist.__dir__())

UserWarning: n-coordinate 2 seems to fall outside lattice

In [ ]:
fept = read("./FePt.cif")

In [50]:
target_atoms.cell

Cell([38.49, 38.49, 37.0])

In [51]:
atoms.cell = target_atoms.cell

In [52]:
write("test.xyz", aft_atoms)

In [53]:
target_repeat = np.array([repeat] * 3) * cell_ratio
target_atoms = fept.repeat(target_repeat.astype(int))

In [57]:
def inner_coord_number(df, i):
    # TODO Tomo need to change to lattice site name
    df["inner_coord_number"] = i
    return df

def make_dataframe(atoms, repeat):
    df = pd.DataFrame(np.concatenate([atoms.get_scaled_positions() * repeat,
                                      (atoms.get_scaled_positions() * repeat) % 0.9999],
                                      axis=1),
                        columns=[f"{i}_{j}" for j in ["cell", "inner"] for i in list("xyz")])
    
    df[["x_cell", "y_cell", "z_cell"]] = df[["x_cell", "y_cell", "z_cell"]].applymap(lambda x : int(x))
    df[["x_inner", "y_inner", "z_inner"]] = df[["x_inner", "y_inner", "z_inner"]].applymap(lambda x : round(x, 3))
    df["types"] = atoms.symbols
    df["types"] = df["types"].map(lambda x : x.lower())

    box = []
    k = 1
    for i, j in df.groupby(by = ["x_inner", "y_inner", "z_inner"
                                 ]):
        tmp_df = inner_coord_number(j, k)
        box.append(tmp_df)
        k += 1
    df = pd.concat(box)

    return df

In [58]:
df_model = make_dataframe(atoms, repeat)
df_target = make_dataframe(target_atoms, repeat)
merged = pd.merge(df_model, df_target, on=["x_cell", "y_cell", "z_cell", "x_inner", "y_inner", "z_inner", "inner_coord_number"], how="left")
merged["types_y"].fillna("empty", inplace=True)

In [43]:
for coord, type in zip(merged[["x_cell", "y_cell", "z_cell", "inner_coord_number"]].values,merged["types_y"]):
    print(coord)
    print(type)
    break

[0 0 0 1]
pt


In [44]:
coord.tolist()

[0, 0, 0, 1]

In [61]:
merged

,x_cell,y_cell,z_cell,x_inner,y_inner,z_inner,types_x,inner_coord_number,types_y
0,0,0,0,0.000,0.000,0.000,fe,1,pt
1,0,0,1,0.000,0.000,0.000,fe,1,empty
2,0,0,2,0.000,0.000,0.000,fe,1,pt
3,0,0,3,0.000,0.000,0.000,fe,1,empty
4,0,0,4,0.000,0.000,0.000,fe,1,pt
...,...,...,...,...,...,...,...,...,...
8051,19,18,19,0.002,0.002,0.002,fe,27,fe
8052,19,19,16,0.002,0.002,0.002,fe,27,pt
8053,19,19,17,0.002,0.002,0.002,fe,27,empty
8054,19,19,18,0.002,0.002,0.002,fe,27,pt
